# Scraping the data for IMDB scores and Metacritic Scores

### Using beautiful soup to scrape all the data from the web

In [ ]:
import urllib.request
from bs4 import BeautifulSoup as bs

get=[]

for x in range(1,1,1):

    url="https://www.imdb.com/search/title?year=2015,2015&title_type=feature&sort=moviemeter,asc&page="+str(x)    
    request = urllib.request.Request(url)
    response = urllib.request.urlopen(request)
    data = response.read()
    response.close()

# Create the soup
    soup = bs(data,"html.parser") 

    get.extend(soup.find_all("div",class_="lister-item-content"))
   
    # Print parse tree
    #print(soup.prettify())

### Gets the IMDB Ratings

In [ ]:
ratings=[]
for elements in get:
    if elements.find("div",class_="inline-block ratings-imdb-rating"):
        z=elements.find("div",class_="inline-block ratings-imdb-rating").get_text().strip()
        ratings.append(float(z))
    else:
        ratings.append("No Data")

### Gets the h3 element which has the name in it

In [ ]:
list1=[]
for elements in get:
    if elements.find("h3", class_="lister-item-header"):
        list1.append(elements.find("h3", class_="lister-item-header"))
    else:
        list1.append(0.0)

### Extracts the name from each element of the above list

In [ ]:
names=[]
for elements in list1:
    if elements.find("a"):
        names.append(elements.find("a").get_text().strip())
    else:
        names.append("No Data")

### Gets the metascore for each of the movie

In [ ]:
meta=[]
for elements in get:
    if elements.find("div",class_="inline-block ratings-metascore"):
        z=elements.find("div",class_="inline-block ratings-metascore").get_text()[0:5].strip()
        meta.append(int(z))
    else:
        meta.append(0)

### Creating a pandas dataframe and transferring every element in the list to the datframe

In [ ]:
import pandas as pd
df1 = pd.DataFrame ({"Names" : names , "IMDB" : ratings , "MetaScore" : meta})

In [ ]:
df1.to_csv("IMDB(100 Pages).csv")

### Now we scraped only the 1st 100 pages because the top 100 movies we were looking for were in the 1st 5000 results

## Now there was a PROBLEM 
The names in the ratings dataframe was similar to in the budget and revenue dataframes, but not exactly the same, so we replace the names in budget and revenue dataframes with those matching or simliar in the IMDB dataframe

In [ ]:
import pandas as pd
imdb_meta = pd.read_csv("IMDB(100 Pages).csv")
budget_rev = pd.read_csv("Budget_Rev.csv")

In [ ]:
del imdb_meta['Unnamed: 0']
del budget_rev["Unnamed: 0"]

In [ ]:
budget_rev.columns = ['Names', 'Budget', 'WorldWide_Rev']

### Using difflib for the word matching

In [ ]:
import difflib

In [ ]:
z=[]
for element in budget_rev["Names"]:
    try:
        print(element)
        print("The closes match is ::::::"+difflib.get_close_matches(str(element),imdb_meta['Names'],cutoff=0.97)[0]+"\n")

        z.append(difflib.get_close_matches(str(element),imdb_meta['Names'],cutoff=0.97)[0])
    except:
        print("No Match")
        print("The name remains the same")
        z.append("No Match")

In [ ]:
budget_rev["Names"]=z

In [ ]:
budget_rev = budget_rev[budget_rev.Names!= 'No Match']

In [ ]:
budget_rev.to_csv("Budget_Revenue.csv")

## Now we have a Budget_Revenue file that has the same names as that of the IMDB_Meta Ratings file, so what's simply left is to join them

In [ ]:
Movies=pd.merge(budget_rev,imdb_meta)

In [ ]:
Movies.drop_duplicates("Names")

#### So after dropping the duplicates we have the exact number of columns that our Budget_revenue file had and we select the top 100 from it.